## Portfolio Optimization using Monte Carlo simulations

The following code would run iterative monte carlo simulations to create a sharpe optimised portfolio. For the purpose of this project we have selected BAJFINANCE, AXISBANK and ASIANPAINT. The flow of this project would involve extracting the data corresponding to the aformentioned securities, assigning randomized weights to the securities of our portfolio and working iteratively through those random weights until we find a sharpe optimized portfolio for the instruments we have considered. 

In [98]:
import numpy as np
import pandas as pd
import pathlib
 

symbols={'BAJFINANCE','AXISBANK','ASIANPAINT'} #considered securities
no_securities= len(symbols) 
dataframe: pd.DataFrame = pd.read_csv(r"wall street club task csv file 2.csv")
dataframe= dataframe[['Date','Symbol','Close']] #creating a pandas dataframe to store the data extracted from the csv
dataframe= dataframe.pivot(
    index='Date',
    columns='Symbol',
    values='Close'

)
display(dataframe) #how the pandas dataframe looks like after importing and transposing(pivot)







Symbol,ASIANPAINT,AXISBANK,BAJFINANCE
Date,,,
2000-01-03,381.65,26.70,50.75
2000-01-04,385.55,26.85,48.10
2000-01-05,383.00,26.30,44.60
2000-01-06,377.50,25.95,45.25
2000-01-07,385.70,24.80,42.90
...,...,...,...
2021-04-26,2557.90,700.45,4736.55
2021-04-27,2574.35,699.55,4865.05
2021-04-28,2614.55,708.15,5280.90


##The following code segment finds first the %age daily changes in the portfolio and the sharpe ratio. This is followed by the code for the Monte Carlo simuations which iterate a 1000 times to arrive at the most sharpe optimized portfolio. The number of iterations can be increaased to arrive at better sharpe optimized portfolios.

In [84]:
returns=np.log(1+dataframe.pct_change())
random_weights = np.array(np.random.random(no_securities))
random_weights_new= random_weights/ np.sum(random_weights)
returns_new= np.sum((returns.mean()*random_weights_new)*252)
volat_new= np.sqrt(
    
np.dot(random_weights_new.T, np.dot(returns.cov()*252, random_weights_new)))
sharpe_ratio= (returns_new-0.07)/volat_new


weights_dataframe = pd.DataFrame(data={
'random weights': random_weights,
'random_weights_new' : random_weights_new
     })

metrics_dataframe = pd.DataFrame(data={
    
    'returns': returns_new ,
    'volatility': volat_new,
    'Sharpe ': sharpe_ratio

                                 }, index=[0])

no_securities= len(symbols)
iterations=100
all_weights=np.zeros((iterations,no_securities))
returns_arr= np.zeros(iterations)
volatility_arr= np.zeros(iterations)
sharpe_arr=np.zeros(iterations)

for ind in range(iterations):
    
    weights=0.3+np.array(np.random.random(no_securities)*(0.1-0.0))
    weights=weights/np.sum(weights)
                                   
    for a in weights:
        
        if a>= 0.3:
            all_weights[ind, :] = weights
            returns_arr[ind]= np.sum((returns.mean()*weights)*252)
            volatility_arr[ind]= np.sqrt(np.dot(weights.T, np.dot( returns.cov()*252, weights)))
            sharpe_arr[ind]= (returns_arr[ind]-0.07)/ volatility_arr[ind]
            iterations_data= [returns_arr, volatility_arr, sharpe_arr, all_weights]
            iterations_dataframe= pd.DataFrame(data=iterations_data).T
              
         
      
iterations_dataframe.columns= ['Returns','Volatility','Sharpe','P']       
iterations_dataframe = iterations_dataframe.infer_objects()
print(iterations_dataframe.head())




    
    

    Returns  Volatility    Sharpe                                                  P
0  0.157566    0.388765  0.225241  [0.3026321662404842, 0.3725732273566277, 0.324...
1  0.157074    0.388638  0.224048  [0.32104873230055053, 0.34314556792030115, 0.3...
2  0.151143    0.385085  0.210715  [0.36881845578961997, 0.33732558020935, 0.2938...
3  0.153229    0.385943  0.215651  [0.35267395508933996, 0.33805441002810804, 0.3...
4  0.154920    0.389078  0.218259  [0.3585338845393011, 0.3006968285424046, 0.340...


## Printing the portfolio corresponding to the most sharpe optimized distribution of weights

In [92]:
print(iterations_dataframe.loc[iterations_dataframe['Sharpe'].idxmax()])



Returns                                                0.161161
Volatility                                             0.393446
Sharpe                                                   0.2317
P             [0.29117544736350665, 0.34104209508899297, 0.3...
Name: 92, dtype: object
